In [8]:

from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm= ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input,output):
    memory.save_context({"input": input},{"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("hi i'm 성호 my goal is rich haha","you will definitely make it")
add_message("my dream is make a javis like iron_man","Okay, I'll help you too.")

get_history()

{'history': 'The human introduces themselves as 성호 and shares their goal of becoming rich. The AI responds with encouragement, saying that the human will definitely achieve their goal. The human then shares their dream of creating a JARVIS-like AI, similar to Iron Man. The AI responds by offering to help the human with their dream.'}